In [1]:
%load_ext tensorboard

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers

#define the convnet 
class LeNet:
	@staticmethod
	def build(input_shape, classes):
		model = models.Sequential()
		# CONV => RELU => POOL
		model.add(layers.Convolution2D(20, (5, 5), activation='relu',
			input_shape=input_shape))
		model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
		# CONV => RELU => POOL
		model.add(layers.Convolution2D(50, (5, 5), activation='relu'))
		model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
		# Flatten => RELU layers
		model.add(layers.Flatten())
		model.add(layers.Dense(500, activation='relu'))
		# a softmax classifier
		model.add(layers.Dense(classes, activation="softmax"))
		return model

# network and training
EPOCHS = 5
BATCH_SIZE = 128
VERBOSE = 2
OPTIMIZER = tf.keras.optimizers.Adam()
VALIDATION_SPLIT=0.20
INPUT_SHAPE = (28, 28, 1)
NB_CLASSES = 10  # number of outputs = number of digits

In [2]:
# data: shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

# reshape
X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))

# normalize
X_train, X_test = X_train / 255.0, X_test / 255.0

# cast
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [3]:
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, NB_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NB_CLASSES)

# initialize the optimizer and model
model = LeNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 20)        520       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 50)          25050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 500)               400500    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5

In [4]:
import os
import datetime

logdir = os.path.join("logs_leNet_CNN_mnist", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb_callbacks = [tf.keras.callbacks.TensorBoard(log_dir = logdir)]

# fit 
history = model.fit(X_train, y_train, 
		batch_size=BATCH_SIZE, epochs=EPOCHS, 
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT,
		callbacks=tb_callbacks)

score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 - 4s - loss: 0.2098 - accuracy: 0.9391 - val_loss: 0.0732 - val_accuracy: 0.9769
Epoch 2/5
48000/48000 - 3s - loss: 0.0562 - accuracy: 0.9829 - val_loss: 0.0668 - val_accuracy: 0.9803
Epoch 3/5
48000/48000 - 3s - loss: 0.0382 - accuracy: 0.9879 - val_loss: 0.0400 - val_accuracy: 0.9878
Epoch 4/5
48000/48000 - 4s - loss: 0.0272 - accuracy: 0.9912 - val_loss: 0.0452 - val_accuracy: 0.9868
Epoch 5/5
48000/48000 - 3s - loss: 0.0207 - accuracy: 0.9934 - val_loss: 0.0421 - val_accuracy: 0.9883
10000/10000 - 1s - loss: 0.0334 - accuracy: 0.9893

Test score: 0.03342088338903995
Test accuracy: 0.9893


In [5]:
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

10000/10000 - 1s - loss: 0.0334 - accuracy: 0.9893

Test score: 0.03342088338903995
Test accuracy: 0.9893
